In [176]:
import sys
import os
import datetime
import calendar
import numpy as np
# for a in os.environ:
#     print(a)
sys.path.insert(0,'/home/spencerh/python/hatch_python_utils/')
#sys.path
print(sys.path)

doPlots = 1

['/home/spencerh/python/hatch_python_utils/', '/home/spencerh/python/hatch_python_utils/', '', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/home/spencerh/.ipython', '/home/spencerh/.emacs.d/elpa/ein-20180227.625/']


In [2]:
import omni_utils
omni = omni_utils.load_omniDB()

Opening culled_OMNI_magdata.dat ...


--------------------------------------------------
Date: Fri May  6 16:53:37 2016
User: spencerh
Host: thelonious.dartmouth.edu
--------------------------------------------------
Format: 12
Architecture: x86_64
Operating System: linux
IDL Version: 8.4
--------------------------------------------------
Successfully read 20 records of which:
 - 1 are of type TIMESTAMP
 - 17 are of type VARIABLE
 - 1 are of type VERSION
--------------------------------------------------
Available variables:
 - cone_overclock_gse [<class 'numpy.ndarray'>]
 - cone_overclock_gsm [<class 'numpy.ndarray'>]
 - thetacone_gsm [<class 'numpy.ndarray'>]
 - mag_utc [<class 'numpy.ndarray'>]
 - phiclock_gse [<class 'numpy.ndarray'>]
 - phiclock_gsm [<class 'numpy.ndarray'>]
 - goodmag_goodtimes_i [<class 'numpy.ndarray'>]
 - bz_gse [<class 'numpy.ndarray'>]
 - bxy_over_bz_gse [<class 'numpy.ndarray'>]
 - bxy_over_bz_gsm [<class 'numpy.ndarray'>]
 - bt_gsm [<class 'numpy.ndarray'>]
 - bx [<class 'numpy.ndarray'>]
 - b

In [43]:
kappaRec.t2Str[1]

'23:11:49'

In [24]:
# As seconds since 1970:
print(omni.mag_utc[0])
# As nicely formatted thing:
print(datetime.datetime.fromtimestamp(omni.mag_utc[0]))
# Convert from datetime to UTC
print(datetime.datetime.fromtimestamp(omni.mag_utc[0],tz=datetime.timezone.utc).replace(tzinfo=datetime.timezone.utc).timestamp())
# And then back
print(datetime.datetime.fromtimestamp(838857600.0))

838857600.0
1996-07-31 20:00:00
838857600.0
1996-07-31 20:00:00


In [39]:
# Read in the ol' txtFile
kappaTxtFile='/SPENCEdata/Research/Satellites/FAST/espec_identification/txtOutput/20180312--mono_aurora_streaks__3-9MLT__1000-3999ORB__300-4300ALT__minTStreak_sec_60.txt'
kappaFmt=np.dtype([('tmpOrb',np.int32),('MLT',np.float64),('ILAT',np.float64),('ALT',np.float64),('t1Str',np.unicode_,19),('t2Str',np.unicode_,8),('t_streakLen',np.int32),('nPts',np.int32),('dt_avg',np.float64),('avg_current',np.float64)])
kappaInfo = np.genfromtxt(kappaTxtFile,dtype=kappaFmt,skip_header=1)
kappaRec = np.rec.array(kappaInfo)

In [171]:
t2Str = kappaRec.t2Str
t1Str = kappaRec.t1Str

# See if we can do substrings
# this = [x[0:11] for x in t1Str]
# print(this)
#this?

# Yup, now combine 'em all
this = ["{}{}".format(t1_[0:11],t2_) for t1_, t2_ in zip(t1Str,t2Str)]

# Print 'em for kicks
# for t1, t2 in zip(t1Str,this):
#     print(t1,", ",t2)

# And convert to datetime objects
t1s=[datetime.datetime.strptime(t1,'%Y-%m-%d/%H:%M:%S') for t1 in t1Str]
t2s=[datetime.datetime.strptime(t2,'%Y-%m-%d/%H:%M:%S') for t2 in this]

# Does it all work? Looks like the answer is 'yes'
# for t1,t2,bros in zip(t1s,t2s,kappaRec.t_streakLen):
#     print(t1,",",t2,",",t2-t1,",",bros)

# Times before and after
# tIncrem=datetime.timedelta(hours=1)
# for t1 in t1s:
#    print(t1-tIncrem,", ",t1+tIncrem)


In [172]:
#t1s.replace(tzinfo=datetime.timezone.utc)
t1Upp = [t1.replace(tzinfo=datetime.timezone.utc) for t1 in t1s]
t2Upp = [t2.replace(tzinfo=datetime.timezone.utc) for t2 in t2s]
t1s   = t1Upp
t2s   = t2Upp

t1UTC = [t1.timestamp() for t1 in t1s]
t2UTC = [t2.timestamp() for t2 in t2s]

In [182]:
########################################
# Set up plots
########################################
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, DateFormatter
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
# Set font sizes for things
legFontSize = 20
tickFontSize=25
titleFontSize=30
matplotlib.rc('xtick',labelsize=tickFontSize)
matplotlib.rc('ytick',labelsize=tickFontSize)
matplotlib.rc('axes',titlesize=titleFontSize)

# Set up ticks
ymajorLocator = MultipleLocator(1)
yminorLocator = MultipleLocator(0.25)

xAxisHours = MinuteLocator(interval=30)   # every hour
xAxisMinutes = MinuteLocator(interval=5)  # every minute
xAxisHoursFmt = DateFormatter('%H:%M')


In [183]:
########################################
# Test a plot
########################################
tIncremSeconds = 60*60
outDir='/SPENCEdata/Research/Satellites/FAST/kappa_dists/plots/20180313/'
outFPref = 'dawnsector_kappa__orb'
outFSuff = '__SW_conds.png'

# k=0
if 'doPlots' in globals() and doPlots == 1:
    for k in range(0,kappaRec.tmpOrb.size-1):

        # tmpInds = ((omni.mag_utc > (t1UTC[k]-tIncremSeconds)) & (omni.mag_utc < (t2UTC[k]+tIncremSeconds)))
        # print(omni.mag_utc[tmpInds].size)
        tmpIndStart=np.argmin(np.abs(omni.mag_utc-(t1UTC[k]-tIncremSeconds)))
        tmpIndStop=np.argmin(np.abs(omni.mag_utc-(t2UTC[k]+tIncremSeconds)))
        print("[{:02}] Orbit {} (inds {}:{})".format(k,kappaRec.tmpOrb[k],tmpIndStart,tmpIndStop))
        if tmpIndStart == tmpIndStop:
            print("Can't do it!")
            continue
        bx=omni.bx[tmpIndStart:tmpIndStop]
        by=omni.by_gsm[tmpIndStart:tmpIndStop]
        bz=omni.bz_gsm[tmpIndStart:tmpIndStop]
        time=[datetime.datetime.fromtimestamp(tid,tz=datetime.timezone.utc) for tid in omni.mag_utc[tmpIndStart:tmpIndStop]]
        dates = matplotlib.dates.date2num(time)
        fig, ax = plt.subplots(figsize=(20,13))

        # Plot the data
        ax.plot_date(dates, bx, '-',linewidth=3)
        ax.plot_date(dates, by, '-',linewidth=3)
        ax.plot_date(dates, bz, '-',linewidth=3)
    
        # format the ticks
        ax.yaxis.set_minor_locator(yminorLocator)
        ax.yaxis.set_major_locator(ymajorLocator)

        ax.xaxis.set_major_locator(xAxisHours)
        ax.xaxis.set_major_formatter(xAxisHoursFmt)
        ax.xaxis.set_minor_locator(xAxisMinutes)
        # ax.set_xlim(time[0], time[-1])
        # ax.autoscale_view()

        # Grid and legend
        ax.grid(which='minor',axis='both')
        ax.legend(['Bx','By (GSM)','Bz (GSM)'],fontsize=legFontSize)

        # Title
        ax.set_title('Orbit {} ('.format(kappaRec.tmpOrb[k])+time[0].strftime('%Y-%m-%d')+')')
        ax.axvline(x=t1s[k],linestyle='dashed',linewidth=2.1,color='r')
        ax.axvline(x=t2s[k],linestyle='dashed',linewidth=2.1,color='r')
        outFName='{:02}__'.format(k)+outFPref+'{}'.format(kappaRec.tmpOrb[k])+outFSuff
        print("Saving to {} ...".format(outFName))
        plt.savefig(outDir+outFName)
        plt.close()

[00] Orbit 2585 (inds 335796:335919)
Saving to 00__dawnsector_kappa__orb2585__SW_conds.png ...


[01] Orbit 1217 (inds 171195:171318)
Saving to 01__dawnsector_kappa__orb1217__SW_conds.png ...


[02] Orbit 1368 (inds 189803:189925)
Saving to 02__dawnsector_kappa__orb1368__SW_conds.png ...


[03] Orbit 2487 (inds 323367:323483)
Saving to 03__dawnsector_kappa__orb2487__SW_conds.png ...


[04] Orbit 2597 (inds 337240:337347)
Saving to 04__dawnsector_kappa__orb2597__SW_conds.png ...


...

[37] Orbit 1991 (inds 262282:262394)
Saving to 37__dawnsector_kappa__orb1991__SW_conds.png ...


[38] Orbit 1952 (inds 257220:257335)
Saving to 38__dawnsector_kappa__orb1952__SW_conds.png ...


[39] Orbit 3837 (inds 487710:487816)
Saving to 39__dawnsector_kappa__orb3837__SW_conds.png ...


[40] Orbit 1238 (inds 173972:174026)
Saving to 40__dawnsector_kappa__orb1238__SW_conds.png ...


[41] Orbit 3815 (inds 485191:485312)
Saving to 41__dawnsector_kappa__orb3815__SW_conds.png ...


In [166]:
time[0]

datetime.datetime(1996, 12, 13, 21, 46, tzinfo=datetime.timezone.utc)

In [111]:
tIncremSeconds = 60*60
for k in range(0,kappaRec.tmpOrb.size-1):
    print("Orbit {}".format(kappaRec.tmpOrb[k]))
    # tmpInds = ((omni.mag_utc > (t1UTC[k]-tIncremSeconds)) & (omni.mag_utc < (t2UTC[k]+tIncremSeconds)))
    # print(omni.mag_utc[tmpInds].size)
    tmpIndStart=np.argmin(np.abs(omni.mag_utc-(t1UTC[k]-tIncremSeconds)))
    tmpIndStop=np.argmin(np.abs(omni.mag_utc-(t2UTC[k]+tIncremSeconds)))
    print("{}:{}".format(tmpIndStart,tmpIndStop))

Orbit 2585
336027:336150
Orbit 1217
171493:171616
Orbit 1368


190028:190130
Orbit 2487
323595:323715
Orbit 2597
337389:337510
Orbit 1607


215000:215113
Orbit 1294
180911:181033
Orbit 2640
342280:342397
Orbit 2347


306438:306556
Orbit 1152
164262:164262
Orbit 2379
309965:310087
Orbit 3956


502936:503058
Orbit 2586
336151:336253
Orbit 2390
311407:311514
Orbit 2685


...

233431:233550
Orbit 3592
459322:459443
Orbit 1433
195573:195573
Orbit 3991


507639:507760
Orbit 3783
481858:481977
Orbit 1293
180780:180901
Orbit 3785


482109:482228
Orbit 1152
164262:164262
Orbit 1991
262569:262684
Orbit 1952


257512:257633
Orbit 3837
487928:488047
Orbit 1238
174026:174027
Orbit 3815


485395:485496
